In [ ]:
#| default_exp mocking.functions

In [ ]:
#| export
import inspect
from pymoq.core import AnyCallable

In [ ]:
#| hide
from fastcore.test import test_fail

# Mocking functions
> From function object to function mock

## Checking arguments against original signature

For a given argument list, we want to know whether the original function can be called with that set of parameters. This can be done in two steps:
1. Extract the signature
2. Try to bind the argument list against the signature. This throws an exception if the arguments can not be matched against the signature

In [ ]:
def f(a: int, b: str, c:str|None =None) -> None:
    pass

Successful bind:

In [ ]:
inspect.signature(f).bind(1, "1", "2")

<BoundArguments (a=1, b='1', c='2')>

Unsuccessful bind:

In [ ]:
try:
    inspect.signature(f).bind(1)
except Exception as e:
    print(e)

missing a required argument: 'b'


In [ ]:
#| export
class FunctionMock:
    "Mocks a function object based on its signature"
    def __init__(self, func: AnyCallable):
        self._func = func
        self._signature = inspect.signature(self._func)
        
    def arguments_valid(self, *args, **kwargs) -> bool:
        "Given an arbitrary argument list (both positional and keyword arguments), returns True if the mocked function could be called with those arguments"
        try:
            self._signature.bind(*args, **kwargs)
            return True
        except:
            return False

In [ ]:
mock = FunctionMock(f)

Successful binds:

In [ ]:
assert mock.arguments_valid(1, "1", "2")
assert mock.arguments_valid(1, "1")
assert mock.arguments_valid(1, "1", c="2")
assert mock.arguments_valid(a=1, b="1", c="2")

Note that the types are not checked with this:

In [ ]:
assert mock.arguments_valid(1, 1)

Unsuccessful binds might be:

In [ ]:
assert not mock.arguments_valid(1) # too few arguments
assert not mock.arguments_valid(1,2,3,4) # too many arguments
assert not mock.arguments_valid(1,2,d=3) # unknown argument name

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()